In [1]:
from bs4 import BeautifulSoup
import pandas as pd
import requests

In [55]:
url = "https://www.goodreads.com/list/show/19.Best_for_Book_Clubs"

In [3]:
html = requests.get(url).text

In [4]:
soup = BeautifulSoup(html, 'html.parser')

In [8]:
soup.find_all('a', class_='bookTitle')[0].text.strip()

'The Help'

In [9]:
soup.find_all('a', class_='authorName')[0].text.strip()

'Kathryn Stockett'

In [36]:
#Ratings
import re

soup.find_all('span', class_='minirating')[0].text.strip()[0:4]


pattern = re.compile(r'/d./d{2}')
re.findall(pattern, soup.find_all('span', class_='minirating')[1])

[]

In [89]:
#Loop
import re

def book(soup):
    book = [i.text.strip() for i in soup.find_all('a', class_='bookTitle')]
    author = [i.text.strip() for i in soup.find_all('a', class_='authorName')]
    
    ratings = []
    pattern = re.compile(r'\d.\d{2}')
    for i in soup.find_all('span', class_='minirating'):
        ratings.append(re.findall(pattern, i.text)[0])
    
    return pd.DataFrame({'Book':book, 'Author':author, 'Rating':ratings})

In [90]:
book(soup)

,Book,Author,Rating
0,"Slammed (Slammed, #1)",Colleen Hoover,4.25
1,Kitchen Confidential: Adventures in the Culina...,Anthony Bourdain,4.08
2,"Anna Dressed in Blood (Anna, #1)",Kendare Blake,3.91
3,"Frostbite (Vampire Academy, #2)",Richelle Mead,4.26
4,Blink: The Power of Thinking Without Thinking,Malcolm Gladwell,3.94
...,...,...,...
95,Hotblooded,Erin Nicholas,3.89
96,The Other Side,Faraaz Kazi,3.57
97,"Liliana (Liliana, #1)",Neva Squires-Rodriguez,3.73
98,The Republic,Plato,3.95


In [91]:
#For N Pages

In [92]:
stub = 'https://www.goodreads.com/list/show/19.Best_for_Book_Clubs?page={}'

In [93]:
from tqdm import tqdm
import time

data = []

for i in tqdm(range(1, 11)):
    url = stub.format(i)
    html = requests.get(url).text
    time.sleep(2)
    soup = BeautifulSoup(html, 'html.parser')
    data.append(book(soup))

100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [01:09<00:00,  6.91s/it]


In [67]:
df = pd.concat(data, axis = 0)
df

,Book,Author,Rating
0,Kathryn Stockett,Kathryn Stockett,4.46
1,Khaled Hosseini,Khaled Hosseini,4.31
2,Sara Gruen,Sara Gruen,4.09
3,Markus Zusak,Markus Zusak,4.38
4,Harper Lee,Harper Lee,4.28
...,...,...,...
95,Erin Nicholas,Erin Nicholas,3.89
96,Faraaz Kazi,Faraaz Kazi,3.57
97,Neva Squires-Rodriguez,Neva Squires-Rodriguez,3.73
98,Plato,Plato,3.95


In [68]:
df['Rating'].astype('float64')

ValueError: could not convert string to float: 'real'